In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import KY_utils.KY_interpretable_functions as interpret
import KY_utils.KY_RiskSLIM as slim
import KY_utils.KY_stumps as stumps

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\KY Model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['drug_six_month'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['drug_six_month'].values

In [3]:
#### CART
depth = [8,9,10]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, seed=816)

### EBM
estimators = [40]
depth = [2]
learning_rate = [0.1]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, seed=816)

In [5]:
cart_summary['FL_score'], ebm_summary['FL_score']

(0.607320540156361, 0.5833333333333334)

### Lasso Stumps

In [6]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['drug_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['drug_six_month'].values

## columns 
cols = KY_X.columns

In [8]:
single_stump_model = stumps.stump_model(KY_X, KY_Y, FL_X, FL_Y, 0.004, cols, 816)
single_stump_model['features']

['sex1',
 'age_at_current_charge18.0',
 'age_at_current_charge19.0',
 'age_at_current_charge21.0',
 'age_at_current_charge30.0',
 'age_at_current_charge35.0',
 'age_at_current_charge41.0',
 'age_at_current_charge46.0',
 'age_at_current_charge47.0',
 'age_at_current_charge49.0',
 'age_at_current_charge70.0',
 'p_arrest1',
 'p_arrest2',
 'p_arrest3',
 'p_arrest4',
 'p_arrest5',
 'p_arrest6',
 'p_arrest7',
 'p_charges1',
 'p_charges2',
 'p_charges5',
 'p_charges6',
 'p_violence1',
 'p_felony1',
 'p_felony2',
 'p_weapon1',
 'p_misdeassult1',
 'p_traffic1',
 'p_drug1',
 'p_drug2',
 'p_drug3',
 'p_pending_charge1',
 'p_probation1',
 'p_probation2',
 'p_incarceration1',
 'one_year1',
 'five_year1']

In [9]:
c_grid={'C': [0.001, 0.002, 0.003, 0.004]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [10]:
stumps_summary['best_params'], stumps_summary['FL_score'], stumps_summary['auc_diffs']

({'C': 0.004}, 0.5864188672024493, 0.0012637371643232642)

### RiskSLIM

In [11]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['drug_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['drug_six_month'].values

## columns 
cols = KY_X.columns

In [39]:
## train on best param chosen by Lasso Stumps from above
single_stump_model = stumps.stump_model(KY_X, KY_Y, FL_X, FL_Y, 
                                        0.0015, 
                                        cols, 
                                        816)
selected_features = single_stump_model['features']
len(selected_features)

19

In [40]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_KY_X.insert(0, '(Intercept)', 1)

In [41]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'drug_six_month', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=1000,
                                max_offset=100,
                                c=1e-3, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
11/13/19 @ 01:25 PM | 243 rows in lookup table
11/13/19 @ 01:25 PM | ------------------------------------------------------------
11/13/19 @ 01:25 PM | runnning initialization procedure
11/13/19 @ 01:25 PM | ------------------------------------------------------------
11/13/19 @ 01:25 PM | CPA produced 2 cuts
11/13/19 @ 01:25 PM | running naive rounding on 6 solutions
11/13/19 @ 01:25 PM | best objective value: 0.5429
11/13/19 @ 01:25 PM | rounding produced 5 integer solutions
11/13/19 @ 01:25 PM | best objective value is 0.5524
11/13/19 @ 01:25 PM | running sequential rounding on 6 solutions
11/13/19 @ 01:25 PM | best objective value: 0.5429
11/13/19 @ 01:25 PM | sequential rounding produced 3 integer solutions
11/13/19 @ 01:25 PM | best objective value: 0.2899
11/13/19 @ 01:25 PM | polishing 8 solutions
11/13/19 @ 01:25 PM | best objective value: 0.2899
11/13/19 @ 01:25 PM | polishing produced 5 integer solutions
11/13/19 @ 0

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/13/19 @ 01:25 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1684.
Tried aggregator 1 time.
Reduced MIP has 40 rows, 42 columns, and 117 nonzeros.
Reduced MIP has 19 binaries, 21 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

 146694 58001        cutoff              0.1654        0.1609   926223    2.77%          rho_11 U 146694  34640     20
 149807 59115        0.1645    17        0.1654        0.1609   942609    2.74%           rho_8 U 149807 149805     22
 152922 60139        0.1631    15        0.1654        0.1609   958633    2.71%           rho_2 U 152922 152921     29
 156025 60995        0.1649    14        0.1654        0.1610   975608    2.68%        alpha_18 D 156025 156024     27
 159380 62152        0.1624    16        0.1654        0.1610   991392    2.66%          rho_19 U 159380 159379     24
 162407 63056        0.1637     9        0.1654        0.1611  1007434    2.64%          rho_11 U 162407 162406     39
 165490 64188        0.1622    15        0.1654        0.1611  1024218    2.62%           rho_1 U 165490 165489     24
 168753 65314        cutoff              0.1654        0.1611  1039716    2.60%        alpha_17 D 168753 168752     24
 171808 66026        0.1639    13        0.1654 

 500131 125104        cutoff              0.1654        0.1631  2764098    1.41%         alpha_6 D 500131 500130     37
 511461 124847        cutoff              0.1654        0.1632  2820854    1.37%           rho_4 U 511461 125091     27
 524367 126782        cutoff              0.1654        0.1632  2878848    1.36%           rho_0 D 524367 524365     31
 537134 128709        0.1637    14        0.1654        0.1632  2939332    1.34%           rho_3 U 537134 190866     27
 548957 128964        0.1636    15        0.1654        0.1633  2995744    1.32%           rho_3 U 548957 105291     21
Elapsed time = 182.39 sec. (168117.16 ticks, tree = 36.12 MB, solutions = 4)
 560883 129529        0.1646     9        0.1654        0.1633  3049973    1.29%           rho_0 D 560883 560882     42
 572410 131183        0.1637     9        0.1654        0.1633  3101988    1.27%          rho_14 U 572410 572409     27
 583593 130528        cutoff              0.1654        0.1634  3153332    1.24%   

CPXPARAM_MIP_Tolerances_LowerCutoff              0.15838048016212372
CPXPARAM_MIP_Tolerances_UpperCutoff              0.16805997251087343


...

 150000 40024        cutoff              0.1656        0.1621   834996    2.11%          rho_12 D 150000   9285     27
 153082 40275        cutoff              0.1656        0.1622   847807    2.07%         alpha_1 U 153082 153080     36
 156224 40600        cutoff              0.1656        0.1622   860625    2.03%           rho_3 D 156224  11493     31
 159238 40723        0.1641    10        0.1656        0.1623   873024    1.99%          rho_19 D 159238 159237     36
 162196 40993        cutoff              0.1656        0.1624   885169    1.95%          rho_16 D 162196 162195     38
 165220 41321        0.1635    11        0.1656        0.1624   897377    1.91%           rho_5 U 165220 165219     37
 168266 41488        0.1654     1        0.1656        0.1625   909262    1.87%           rho_0 D 168266 168264     35
 171454 41813        0.1640     8        0.1656        0.1625   921089    1.84%           rho_8 U 171454 171452     34
 174543 42047        0.1638    12        0.1656 

11/13/19 @ 01:38 PM | 243 rows in lookup table
11/13/19 @ 01:38 PM | ------------------------------------------------------------
11/13/19 @ 01:38 PM | runnning initialization procedure
11/13/19 @ 01:38 PM | ------------------------------------------------------------
11/13/19 @ 01:38 PM | CPA produced 2 cuts
11/13/19 @ 01:38 PM | running naive rounding on 6 solutions
11/13/19 @ 01:38 PM | best objective value: 0.5410
11/13/19 @ 01:38 PM | rounding produced 5 integer solutions
11/13/19 @ 01:38 PM | best objective value is 0.4722
11/13/19 @ 01:38 PM | running sequential rounding on 6 solutions
11/13/19 @ 01:38 PM | best objective value: 0.5410
11/13/19 @ 01:38 PM | sequential rounding produced 3 integer solutions
11/13/19 @ 01:38 PM | best objective value: 0.2899
11/13/19 @ 01:38 PM | polishing 8 solutions
11/13/19 @ 01:38 PM | best objective value: 0.2899
11/13/19 @ 01:38 PM | polishing produced 4 integer solutions
11/13/19 @ 01:38 PM | initialization produced 10 feasible solutions
11/

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/13/19 @ 01:38 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1683.
Tried aggregator 1 time.
Reduced MIP has 40 rows, 42 columns, and 117 nonzeros.
Reduced MIP has 19 binaries, 21 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

 135708 49561        cutoff              0.1654        0.1605   865381    2.94%           rho_2 D 135708 135707     27
 138607 50346        cutoff              0.1654        0.1605   881464    2.92%          rho_13 D 138607 138606     14
 141473 51059        0.1618    15        0.1654        0.1606   898500    2.90%         alpha_1 D 141473 141472     24
 144483 51787        cutoff              0.1654        0.1606   913670    2.87%           rho_3 U 144483 144480     22
 147388 52564        cutoff              0.1654        0.1607   929210    2.84%          rho_18 U 147388 147386     40
 150300 53222        0.1621    14        0.1654        0.1607   945300    2.82%           rho_3 D 150300 150298     18
 153295 54163        0.1624    16        0.1654        0.1607   960552    2.80%           rho_2 U 153295 153294     30
 156299 55100        cutoff              0.1654        0.1608   975614    2.79%        alpha_16 D 156299 156298     40
 159230 56387        0.1612    16        0.1654 

 454317 123207        0.1646    12        0.1654        0.1626  2482051    1.70%           rho_8 N 454317 454315     20
 464644 123554        cutoff              0.1654        0.1626  2534114    1.66%           rho_8 D 464644 261374     28
 475345 124559        0.1644    15        0.1654        0.1627  2590235    1.63%           rho_0 U 475345 475343     26
 486154 124899        0.1636    12        0.1654        0.1627  2641188    1.59%          rho_13 U 486154 486153     31
 497220 126769        0.1648     9        0.1654        0.1628  2689477    1.58%         alpha_1 D 497220 497219     35
 507570 127005        0.1630    14        0.1654        0.1628  2744317    1.55%           rho_8 D 507570 465626     25
Elapsed time = 116.89 sec. (168120.77 ticks, tree = 35.82 MB, solutions = 6)
 518010 127391        cutoff              0.1654        0.1629  2798314    1.52%         alpha_1 U 518010 518008     36
 528884 129404        0.1632    14        0.1654        0.1629  2849494    1.50%   

In [42]:
riskslim_summary['FL_score'], np.mean(riskslim_summary['KY_validation_score'])

(0.5656914602810126, 0.7374991756554584)

In [43]:
#### save results
summary_drug6_ky_inter_model = {"CART": cart_summary,
                                "EBM": ebm_summary,
                                "Stumps": stumps_summary,
                                "RiskSLIM": riskslim_summary}

In [44]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\KY Model\\interpretable\\Six Month\\"
results = [["Drug",             
            round(cart_summary['FL_score'],3), 
            round(ebm_summary['FL_score'], 3), 
            round(stumps_summary['FL_score'],3), 
            round(riskslim_summary['FL_score'], 3)]]
with open(path + 'Six Month KY_inter_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)